# Zpracování HTML a JSON souborů

Více informací o HTML a JSON najdete třeba na wikipedii:
* <a href="https://en.wikipedia.org/wiki/JSON">JavaScript Object Notation</a>
* <a href="https://en.wikipedia.org/wiki/HTML">HyperText Markup Language (HTML)</a>

## SAX a DOM
Dva základní přístupy k parsování souborů (používá se hlavně ve spojení s XML a JSON)

* SAX (Simple API for XML) - skenování obsahu souborů za pochodu, nic se neskladuje do paměti

* DOM (Document Object Model) - vytvoření stromové reprezentace obsahu v paměti


## Základní podpora parsování HTML v Pythonu - HTMLParser 

In [1]:
sample_html =  """
<html>
    <head>
        <title>Test</title>
    </head>
    <body>
        <h1>Heading!</h1>
        <p class="major_content">Some content.</p>
        <p class="minor_content">Some other content.</p>
    </body>
</html>
"""

Následuje příklad jak prohledávat (parsovat) HTML výše:

In [2]:
from html.parser import HTMLParser

class TestHTMLParser(HTMLParser):
    
    def handle_starttag(self, tag, attrs):
        print("Tag start:", tag)

    def handle_endtag(self, tag):
        print("Tag end:", tag)

    def handle_data(self, data):
        print("Tag data:", data)
    
parser = TestHTMLParser(convert_charrefs=True)
parser.feed(sample_html)

Tag data: 

Tag start: html
Tag data: 
    
Tag start: head
Tag data: 
        
Tag start: title
Tag data: Test
Tag end: title
Tag data: 
    
Tag end: head
Tag data: 
    
Tag start: body
Tag data: 
        
Tag start: h1
Tag data: Heading!
Tag end: h1
Tag data: 
        
Tag start: p
Tag data: Some content.
Tag end: p
Tag data: 
        
Tag start: p
Tag data: Some other content.
Tag end: p
Tag data: 
    
Tag end: body
Tag data: 

Tag end: html
Tag data: 



Následuje příklad, kde je hledán pouze obsah paragrafu, který má atribut s hodnotou: `major_content`.

In [3]:
class Test2HTMLParser(HTMLParser):

    def __init__(self):
        HTMLParser.__init__(self, convert_charrefs=True)
        self.recording = False
    
    def handle_starttag(self, tag, attrs):
        if tag == "p" and "major_content" in dict(attrs).values():
            self.recording = True

    def handle_endtag(self, tag):
        self.recording = False

    def handle_data(self, data):
        if self.recording:
            print(data)

parser2 = Test2HTMLParser()
parser2.feed(sample_html)

Some content.


## DOM parsování pomocí ElementTree XML API

Tento nástroj je určen pro XML, ale je možné ho využít i pro HTML. Není třeba instalovat.

In [4]:
import xml.etree.ElementTree as ET

tree = ET.fromstring(sample_html)

for child1 in tree:
    print(child1.tag)
    for child2 in child1:
        print("\t", child2.tag, "-", child2.text)

head
	 title - Test
body
	 h1 - Heading!
	 p - Some content.
	 p - Some other content.


Následuje příklad, kde je hledán pouze obsah paragrafu, který má atribut s hodnotou: `major_content`.

In [5]:
import xml.etree.ElementTree as ET

tree = ET.fromstring(sample_html)
        
tree.findall("./body/p[@class='major_content']")[0].text

'Some content.'

## BeautifulSoup knihovny pro parsování HTML
BeautifulSoup je knihovna, kterou je možno považovat za standard pro parsování HTML v Pythonu. Knihovnu je možné instalovat přes PIP:

`pip install bs4`

In [6]:
from bs4 import BeautifulSoup

# path to data
path = "data/example1.html"

# template for printing the output
sentence = "{} {} is {} years old."

# load data
with open(path, 'r') as datafile:
    sample_html = datafile.read()

# create tree
soup = BeautifulSoup(sample_html, "html.parser")

# get title and print it
title = soup.find("title")
print(title.text, "\n")

# select all rows in table
table = soup.find("table",  {"id": "main_table"})
table_rows = table.findAll("tr")  

# iterate over table and print results
for row in table_rows:
    first_name = row.find("td", {"class": "first_name"})
    last_name = row.find("td", {"class": "last_name"})
    age = row.find("td", {"class": "age"})
    if first_name and last_name and age:
        print(sentence.format(first_name.text, last_name.text, age.text))

Example webpage! 

Alice Smith is 31 years old.
Bob Stone is 38 years old.
Narcissus Hyacinth is 34 years old.
Adelmar Egino is 50 years old.


Atributy jednotlivých elemntů jsou přístupné následovně:

In [7]:
print(table.attrs)

{'id': 'main_table'}


## Získání specifické části textu
V případech kdy hledáme se souboru/textu jednu konkrétní informaci, tak není často nutné ho parsovat

In [8]:
sample_html =  """
<html>
    <head>
        <title>Test</title>
    </head>
    <body>
        <h1>Heading!</h1>
        <p class="major_content">Some content. And even more content.</p>
        <p class="minor_content">
            Some other content.
            Numbers related content.
            The important information is, that the key number is 23.
        </p>
    </body>
</html>
"""

Následuje příklad: zjistit jaké číslo je ve větě `the key number is __.` kde `__` reprezentuje hledané číslo.

In [9]:
# unclean way
target_start = sample_html.find("the key number is ") + len("the key number is")
target_end = sample_html[target_start:].find(".") + target_start
print(sample_html[target_start:target_end])

 23


Lepší možnost je použít regex <a href= https://docs.python.org/2/library/re.html>Regex</a>.

In [10]:
# much beter way (with regex)
import re
print(re.search('the key number is (.*).', sample_html).group(1))

23


## Práce s JSON v Pythonu
V Pythonu je knihovna JSON, která umožňuje převod pythonních struktur do a z JSON textu.

### Z Pythonu do JSONu

In [11]:
import json

# sample data
message = [
    {"time": 123, "value": 5},
    {"time": 124, "value": 6},
    {"status": "ok", "finish": [True, False, False]}, 
]

# pack message as json
js_message = json.dumps(message)

# show result
print(type(js_message))
print(js_message)

<class 'str'>
[{"time": 123, "value": 5}, {"time": 124, "value": 6}, {"status": "ok", "finish": [true, false, false]}]


### Z JSONu do Pythonu

In [12]:
# unpack message
message = json.loads(js_message)

# show result
print(type(message))
print(message)

<class 'list'>
[{'time': 123, 'value': 5}, {'time': 124, 'value': 6}, {'status': 'ok', 'finish': [True, False, False]}]
